In [ ]:
import pandas as pd
import os
os.environ["WANDB_DISABLED"]="true"

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import transformers
print(transformers.__version__)

4.25.1


In [ ]:
model_checkpoint = "bigscience/mt0-small"

# Load the dataset

In [ ]:
import json

with open('/content/ob-loose-jun28-sm.jsonl', 'r') as json_file:
    jsonl = json_file.readlines()

In [ ]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/610320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-13-b4856c86bc22>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
# Format the data in the required format
data = [{'sequences':json.loads(t)} for t in jsonl]

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1)

validation, test = train_test_split(test, test_size=0.5)


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1)

validation, test = train_test_split(test, test_size=0.5)


In [ ]:
train_df.to_json(path_or_buf='/content/ob-loose-jun28-sm_train.jsonl', orient='records', lines=True)
validation_df.to_json(path_or_buf='/content/ob-loose-jun28-sm_validation.jsonl', orient='records', lines=True)
test_df.to_json(path_or_buf='/content/ob-loose-jun28-sm_test.jsonl', orient='records', lines=True)

In [ ]:
from datasets import load_dataset
base_path = '/content/ob-loose-jun28-sm_'
raw_datasets = load_dataset("json", data_files={"train": base_path + "train.jsonl", "validation": base_path + "validation.jsonl", "test": base_path + "test.jsonl"})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-7b8709c1db52b523/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sequences'],
        num_rows: 907
    })
    validation: Dataset({
        features: ['sequences'],
        num_rows: 50
    })
    test: Dataset({
        features: ['sequences'],
        num_rows: 51
    })
})

In [ ]:
raw_datasets["train"][0]

{'sequences': {'prompt': 'Rephrase and join the sentences to remove repetition and sound more human without changing the wording and semantics.\nThe only exception is that you are allowed to rephrase from the user query the <|NOT_SURE|> parts.\n\n###\n\nQuestion: who is the new CEO of Consumer Goods X?\nRobotic Answer: <|Tricia Nolly is currently the President and Chief Executive Officer of Consumer Goods X who has 20 years of experience in handling European food and beverage brands.|>\nHuman Answer:',
  'completion': ' Tricia Nolly is currently the President and Chief Executive Officer of Consumer Goods X and has 20 years of experience in handling European food and beverage brands.\n'}}

## Preprocessing

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/430 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source = "prompt"
target = "completion"

def preprocess_function(examples):
    inputs = [prefix + ex[source] for ex in examples["sequences"]]
    targets = [ex[target] for ex in examples["sequences"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[788, 59990, 305, 16853, 287, 101654, 299, 288, 19303, 584, 110426, 305, 17882, 1097, 6910, 259, 5767, 259, 27837, 287, 2274, 347, 305, 259, 214498, 263, 260, 486, 2469, 37709, 339, 533, 521, 418, 259, 39522, 288, 584, 59990, 702, 287, 12394, 259, 35483, 287, 1042, 409, 137292, 290, 172629, 409, 669, 21825, 260, 259, 62569, 69682, 267, 1866, 339, 287, 1546, 31782, 304, 259, 72016, 8204, 263, 959, 291, 33753, 2161, 83817, 267, 1042, 409, 38495, 1729, 653, 38614, 339, 7301, 484, 287, 13976, 305, 259, 19421, 32189, 56361, 304, 259, 72016, 8204, 263, 959, 1866, 1070, 628, 3127, 304, 4842, 281, 63927, 13106, 8314, 305, 330, 68915, 62840, 260, 409, 669, 13752, 83817, 267, 1], [788, 59990, 305, 16853, 287, 101654, 299, 288, 19303, 584, 110426, 305, 17882, 1097, 6910, 259, 5767, 259, 27837, 287, 2274, 347, 305, 259, 214498, 263, 260, 486, 2469, 37709, 339, 533, 521, 418, 259, 39522, 288, 584, 59990, 702, 287, 12394, 259, 35483, 287, 1042, 409, 137292, 290, 172629, 409, 669, 2182

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Fine-tuning the Bloomz model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bigscience--mt0-small/snapshots/a4d96c1fcc9a760510ff4d29be699dab0d02807e/config.json
Model config MT5Config {
  "_name_or_path": "bigscience/mt0-small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 250112
}

loading weights file pyt

In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source}-to-{target}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: sequences. If sequences are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 907
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 342
  Number of trainable parameters = 300176768


OutOfMemoryError: ignored

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/mt0-small-finetuned-prompt-to-completion'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
trainer.save_model("mt0-small-finetuned")

In [ ]:
model_name = 'mt0-small-finetuned'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
src_text = raw_datasets['test'][2]['sequences']['prompt']
tgt_text = raw_datasets['test'][2]['sequences']['completion']
print(src_text)
print(tgt_text)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]